<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/05_crewAI/01_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Agents to Research and Write an Article

In this notebook, we will introduce to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries required are the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI library.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for our agents, we'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for the Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define our Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define our Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create our crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew (1)

**Note**: LLMs can provide different outputs for they same input.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Rise of Artificial Intelligence: Trends, Key Players, and News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its growing impact on various industries
- Mention of the latest trends in AI technology

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning advancements
- Natural Language Processing (NLP) applications
- AI in healthcare, finance, and autonomous vehicles
- Ethical considerations in AI development

2. Key Players in the AI Ind

I now can give a great answer 

Final Answer:
# The Rise of Artificial Intelligence: Trends, Key Players, and News

## Introduction:
Artificial Intelligence (AI) continues to revolutionize various industries with its rapid advancements and innovative applications. From machine learning to natural language processing, AI technology is reshaping the way businesses operate and interact with customers. The latest trends in AI highlight the potential for growth and development in areas such as healthcare, finance, and autonomous vehicles. As we delve deeper into the realm of AI, ethical considerations in its development become increasingly crucial to address.

## Latest Trends in Artificial Intelligence:
Machine Learning and Deep Learning have been at the forefront of AI advancements, enabling systems to learn and improve from experience without explicit programming. Natural Language Processing (NLP) applications have also gained momentum, allowing machines to understand, interpret, and gen

- Display the results of the execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Trends, Key Players, and News

## Introduction:
Artificial Intelligence (AI) continues to revolutionize various industries with its rapid advancements and innovative applications. From machine learning to natural language processing, AI technology is reshaping the way businesses operate and interact with customers. The latest trends in AI highlight the potential for growth and development in areas such as healthcare, finance, and autonomous vehicles. As we delve deeper into the realm of AI, ethical considerations in its development become increasingly crucial to address.

## Latest Trends in Artificial Intelligence:
Machine Learning and Deep Learning have been at the forefront of AI advancements, enabling systems to learn and improve from experience without explicit programming. Natural Language Processing (NLP) applications have also gained momentum, allowing machines to understand, interpret, and generate human language. The integration of AI in healthcare, finance, and autonomous vehicles showcases the transformative impact of this technology on various sectors. However, as AI continues to evolve, ethical considerations surrounding data privacy, bias, and transparency must be carefully navigated.

## Key Players in the AI Industry:
Leading the charge in AI research and development are key players such as Google's AI research division, Google Brain, and Amazon Web Services (AWS) AI services. IBM Watson AI platform and Microsoft Azure AI capabilities have also made significant contributions to the AI landscape. Additionally, startups like OpenAI and DeepMind are pushing the boundaries of AI innovation with groundbreaking solutions and applications. The collaborative efforts of these key players are instrumental in driving the future of AI technology forward.

## Noteworthy News in Artificial Intelligence:
Recent breakthroughs in AI research have paved the way for novel applications and solutions to real-world problems. From enhancing healthcare diagnostics to optimizing financial services, AI is unlocking new possibilities for innovation. Industry partnerships and collaborations in AI development further accelerate the pace of technological advancements, fostering a dynamic ecosystem of growth and discovery. As AI continues to make headlines with its transformative impact, staying informed about the latest news and developments in the field is essential for professionals and enthusiasts alike.

## Audience Analysis:
Professionals in technology, business, and innovation form the target audience for this blog post, seeking to deepen their understanding of AI advancements and industry applications. With a keen interest in the future implications of AI technology, the audience may face challenges in grasping complex concepts and keeping up with rapid industry changes. By providing valuable insights and practical examples, this blog post aims to empower readers to make informed decisions and navigate the evolving landscape of AI with confidence.

## Call to Action:
As we embark on this journey through the world of Artificial Intelligence, I encourage you to stay informed about AI developments by subscribing to industry newsletters or attending relevant conferences. Your thoughts and experiences with AI technology are valuable, so feel free to share them in the comments section below. Let's continue to explore the endless possibilities of AI together and shape a future driven by innovation and collaboration.

## Running the Crew (2)

In [ ]:
topic = "AI Agents"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Agents.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Exploring the World of AI Agents: Trends, Key Players, and News

Outline:
I. Introduction
- Definition of AI Agents
- Importance of AI Agents in various industries
II. Latest Trends in AI Agents
- Machine learning advancements
- Natural language processing capabilities
- Personalization and customization features
III. Key Players in the AI Agents Market
- Google Assistant
- Amazon Alexa
- Apple Siri
- Microsoft Cortana
IV. Noteworthy News on AI Agents
- Recent advancements in AI technology
- Impact o

I now can give a great answer

Final Answer:
# Exploring the World of AI Agents: Trends, Key Players, and News

AI Agents, also known as virtual assistants or chatbots, are revolutionizing the way we interact with technology. These intelligent systems are designed to understand and respond to human language, making them invaluable in various industries such as healthcare, finance, and customer service. The importance of AI Agents lies in their ability to streamline processes, improve efficiency, and enhance user experience.

One of the latest trends in AI Agents is the advancement of machine learning algorithms, allowing these systems to continuously learn and improve their performance. Additionally, natural language processing capabilities have significantly enhanced the ability of AI Agents to understand and respond to complex human language. Personalization and customization features are also on the rise, enabling AI Agents to provide tailored experiences to users based on their pre

In [ ]:
Markdown(result)

# Exploring the World of AI Agents: Trends, Key Players, and News

AI Agents, also known as virtual assistants or chatbots, are revolutionizing the way we interact with technology. These intelligent systems are designed to understand and respond to human language, making them invaluable in various industries such as healthcare, finance, and customer service. The importance of AI Agents lies in their ability to streamline processes, improve efficiency, and enhance user experience.

One of the latest trends in AI Agents is the advancement of machine learning algorithms, allowing these systems to continuously learn and improve their performance. Additionally, natural language processing capabilities have significantly enhanced the ability of AI Agents to understand and respond to complex human language. Personalization and customization features are also on the rise, enabling AI Agents to provide tailored experiences to users based on their preferences and behavior.

Some of the key players in the AI Agents market include Google Assistant, Amazon Alexa, Apple Siri, and Microsoft Cortana. These tech giants have invested heavily in developing their virtual assistant technologies, making them household names in the world of AI. Each of these AI Agents offers unique features and capabilities, catering to different user needs and preferences.

Recent advancements in AI technology have been making headlines, with breakthroughs in areas such as speech recognition, image recognition, and natural language understanding. AI Agents have also been making a significant impact on customer service, with businesses leveraging these systems to provide efficient and personalized support to their customers.

The target audience for AI Agents includes tech-savvy individuals interested in AI technology, businesses looking to implement AI solutions, and marketers seeking to understand AI trends. By staying informed about the latest developments in AI Agents, these individuals can make informed decisions about how to leverage AI technology in their respective fields.

I encourage readers to explore AI Agents further by trying out demos or case studies offered by companies that specialize in AI technology. By experiencing firsthand the capabilities of AI Agents, readers can gain a deeper understanding of how these systems can benefit them personally or professionally.

In conclusion, AI Agents are shaping the future of technology and transforming the way we interact with machines. By staying informed about the latest trends, key players, and news in the world of AI Agents, individuals and businesses can harness the power of AI to drive innovation and growth. 

Resources:
- [Gartner's report on AI market trends](link to report)
- [Forbes article on the impact of AI Agents](link to article)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### just pass "llm" to the agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### just pass "llm" to the agent function
```

### For using Llama locally with **Ollama** and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).